In [1]:
import scipy.io as sio
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import os
from sklearn.preprocessing import MinMaxScaler
import re
import pickle

### 分割数据，按照采样率来分割

In [18]:
home_dir = os.getcwd()
home_dir

'E:\\EEG\\code\\TM\\TS-TCC\\data_preprocessing\\SEED\\data_files'

In [7]:
def split_data(train_data, train_labels, subject_num, output_dir='../../data/FACED/') -> None:
    # 分割数据
    # 先将数据分割为训练集和测试验证集，8：2 的比例
    X_train, X_val_test, y_train, y_val_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)
    # 在将验证集测试集划分为验证集和测试集，1：1的比例
    X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)
    save_path = os.path.join(output_dir + f'subject{subject_num}')
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    dat_dict = dict()
    dat_dict["samples"] = torch.from_numpy(X_train)
    dat_dict["labels"] = torch.from_numpy(y_train)
    torch.save(dat_dict, os.path.join(save_path, "train.pt"))

    dat_dict = dict()
    dat_dict["samples"] = torch.from_numpy(X_val)
    dat_dict["labels"] = torch.from_numpy(y_val)
    torch.save(dat_dict, os.path.join(save_path, "val.pt"))

    dat_dict = dict()
    dat_dict["samples"] = torch.from_numpy(X_test)
    dat_dict["labels"] = torch.from_numpy(y_test)
    torch.save(dat_dict, os.path.join(save_path, "test.pt"))

# 数据集2

In [2]:
with open('sub000.pkl', 'rb') as f:
    data = pickle.load(f)


In [4]:
label_a = pd.read_csv('label.txt', header=None)

FileNotFoundError: [Errno 2] No such file or directory: 'label.txt'

In [3]:
data.shape

(28, 32, 7500)

In [3]:
label_a = np.array(label_a)
print(label_a.shape)
for i in range(label_a.shape[0]):
    if  0<=i<=2:
        label_a[i,0] = 0
    elif 3<=i<=5:
        label_a[i,0] = 1
    elif 6<=i<=8:
        label_a[i,0] = 2
    elif 9<=i<=11:
        label_a[i,0] = 3
    elif 12<=i<=15:
        label_a[i,0] = 4
    elif 16<=i<=18:
        label_a[i,0] = 5
    elif 19<=i<=21:
        label_a[i,0] = 6
    elif 22<=i<=24:
        label_a[i,0] = 7
    elif 25<=i<=27:
        label_a[i,0] = 8
print(label_a)

(28, 1)
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [3]
 [3]
 [3]
 [4]
 [4]
 [4]
 [4]
 [5]
 [5]
 [5]
 [6]
 [6]
 [6]
 [7]
 [7]
 [7]
 [8]
 [8]
 [8]]


In [4]:
data_path = 'E:\\EEG\\datasets\\FACED\\EEG_Features\\DE\\'
faced_data = []
faced_label = []
for file in os.listdir(data_path):
    sub_name = file.split('.')[0]
    sub_num = int(sub_name[3:])
    with open(data_path + file, 'rb') as f:
        data = pickle.load(f)
    all_data = []
    label_all = []
    for i in range(data.shape[2]):
        data_i = data[:,:,i,:]
        all_data.append(data_i)
        label_all.append(label_a)
    train_data = np.concatenate(all_data, axis=0)
    train_labels = np.concatenate(label_all, axis=0).reshape(-1)
    faced_data.append(train_data)
    faced_label.append(train_labels)
faced_all_data = np.concatenate(faced_data, axis=0)
faced_all_label = np.concatenate(faced_label, axis=0)


(103320, 32, 5)

In [8]:
split_data(faced_all_data, faced_all_label, 123, output_dir='../../../data/FACED/')